In [5]:
import pandas as pd
import matplotlib as plt
import sqlite3
import os

In [7]:
#retriing table names from data frame
im_path = r"C:\Users\HP\Desktop\moringa\PHASE1_PROJECT\im.db\im.db"

conn = sqlite3.connect(im_path)
cursor = conn.cursor()

table_name_query = """ SELECT name
                       AS 'Table Names' 
                       FROM sqlite_master 
                       WHERE type='table';
                       """

table_names = pd.read_sql(table_name_query, conn)

print(table_names)

     Table Names
0   movie_basics
1      directors
2      known_for
3     movie_akas
4  movie_ratings
5        persons
6     principals
7        writers


In [9]:
q = """SELECT *
       FROM directors;
    """
df = pd.read_sql(q, conn)

print(df.head())

    movie_id  person_id
0  tt0285252  nm0899854
1  tt0462036  nm1940585
2  tt0835418  nm0151540
3  tt0835418  nm0151540
4  tt0878654  nm0089502


In [6]:
# marging the movie mudgets in csv file and  cleaning them up
budget_one = pd.read_csv("C:\\Users\\HP\\Desktop\\moringa\\PHASE1_PROJECT\\tn.movie_budgets.csv\\tn.movie_budgets.csv")
budget_two = pd.read_csv("C:\\Users\\HP\\Desktop\\moringa\\PHASE1_PROJECT\\bom.movie_gross.csv\\bom.movie_gross.csv")


In [15]:
budget_one.head()
budget_one.tail()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
5777,78,"Dec 31, 2018",Red 11,"$7,000",$0,$0
5778,79,"Apr 2, 1999",Following,"$6,000","$48,482","$240,495"
5779,80,"Jul 13, 2005",Return to the Land of Wonders,"$5,000","$1,338","$1,338"
5780,81,"Sep 29, 2015",A Plague So Pleasant,"$1,400",$0,$0
5781,82,"Aug 5, 2005",My Date With Drew,"$1,100","$181,041","$181,041"


In [26]:
merged_budget = pd.merge(budget_one, budget_two, left_on='movie', right_on='title', how='inner')
merged_budget

,movie,production_budget,domestic_gross_x,worldwide_gross,title,studio,domestic_gross_y,foreign_gross,year
0,Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875",Pirates of the Caribbean: On Stranger Tides,BV,241100000.0,804600000,2011
1,Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963",Avengers: Age of Ultron,BV,459000000.0,946400000,2015
2,Avengers: Infinity War,"$300,000,000","$678,815,482","$2,048,134,200",Avengers: Infinity War,BV,678800000.0,"1,369.5",2018
3,Justice League,"$300,000,000","$229,024,295","$655,945,209",Justice League,WB,229000000.0,428900000,2017
4,Spectre,"$300,000,000","$200,074,175","$879,620,923",Spectre,Sony,200100000.0,680600000,2015
...,...,...,...,...,...,...,...,...,...
1242,Sound of My Voice,"$135,000","$408,015","$429,448",Sound of My Voice,FoxS,408000.0,NaN,2012
1243,Your Sister's Sister,"$120,000","$1,597,486","$3,090,593",Your Sister's Sister,IFC,1600000.0,NaN,2012
1244,The Gallows,"$100,000","$22,764,410","$41,656,474",The Gallows,WB (NL),22800000.0,20200000,2015
1245,A Ghost Story,"$100,000","$1,594,798","$2,769,782",A Ghost Story,A24,1600000.0,NaN,2017


In [19]:
merged_budget.drop_duplicates()

,id,release_date,movie,production_budget,domestic_gross_x,worldwide_gross,title,studio,domestic_gross_y,foreign_gross,year
0,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875",Pirates of the Caribbean: On Stranger Tides,BV,241100000.0,804600000,2011
1,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963",Avengers: Age of Ultron,BV,459000000.0,946400000,2015
2,7,"Apr 27, 2018",Avengers: Infinity War,"$300,000,000","$678,815,482","$2,048,134,200",Avengers: Infinity War,BV,678800000.0,"1,369.5",2018
3,9,"Nov 17, 2017",Justice League,"$300,000,000","$229,024,295","$655,945,209",Justice League,WB,229000000.0,428900000,2017
4,10,"Nov 6, 2015",Spectre,"$300,000,000","$200,074,175","$879,620,923",Spectre,Sony,200100000.0,680600000,2015
...,...,...,...,...,...,...,...,...,...,...,...
1242,68,"Apr 27, 2012",Sound of My Voice,"$135,000","$408,015","$429,448",Sound of My Voice,FoxS,408000.0,NaN,2012
1243,73,"Jun 15, 2012",Your Sister's Sister,"$120,000","$1,597,486","$3,090,593",Your Sister's Sister,IFC,1600000.0,NaN,2012
1244,80,"Jul 10, 2015",The Gallows,"$100,000","$22,764,410","$41,656,474",The Gallows,WB (NL),22800000.0,20200000,2015
1245,86,"Jul 7, 2017",A Ghost Story,"$100,000","$1,594,798","$2,769,782",A Ghost Story,A24,1600000.0,NaN,2017


In [27]:
print(merged_budget.columns)

Index(['movie', 'production_budget', 'domestic_gross_x', 'worldwide_gross',
       'title', 'studio', 'domestic_gross_y', 'foreign_gross', 'year'],
      dtype='object')
